# Imports

In [95]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import soundfile as sf
from scipy.fft import rfft,rfftfreq
import librosa
import librosa.display
import IPython.display as ipd
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pickle
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture

warnings.filterwarnings('ignore')

# Model

### define features extraction function

In [96]:
def mfcc_feature_extractor(audio,sampleRate):
    mfccsFeatures = librosa.feature.mfcc(y=audio, sr=sampleRate, n_mfcc=40)
    mfccsScaledFeatures = np.mean(mfccsFeatures.T,axis=0)
    
    return mfccsScaledFeatures

In [97]:
def contrast_feature_extractor(audio,sampleRate):
    stft = np.abs(librosa.stft(audio))
    contrast = librosa.feature.spectral_contrast(S=stft, sr=sampleRate) 
    contrastScaled = np.mean(contrast.T,axis=0)
    
    return contrastScaled

In [98]:
def tonnetz_feature_extractor(audio,sampleRate):
    tonnetz = librosa.feature.tonnetz(y=librosa.effects.harmonic(audio), sr=sampleRate)
    tonnetzScaled = np.mean(tonnetz.T,axis=0)
    
    return tonnetzScaled

In [99]:
def centroid_feature_extractor(audio,sampleRate):
    centroid = librosa.feature.spectral_centroid(y=audio,sr=sampleRate) 
    centroidScaled = np.mean(centroid.T,axis=0)
    
    return centroidScaled

In [100]:
def features_extractor(file):
    features=[]
    audio, sampleRate = librosa.load(file, res_type='kaiser_fast') 
    mfcc=mfcc_feature_extractor(audio,sampleRate)
    # contrast = contrast_feature_extractor(audio,sampleRate)
    # tonnetz = tonnetz_feature_extractor(audio,sampleRate)
    # centroid = centroid_feature_extractor(audio,sampleRate)
    features.append(mfcc)
    # features[0] = np.concatenate((features[0][0],features[0][1],features[0][2],features[0][3]))
    return features

### define add features function

In [101]:
def add_features(extractedFeatures,dirPath,label):
    for file in os.listdir(dirPath):
        filePath = dirPath+'/'+file
        finalClassLabels=label
        audio, sampleRate = librosa.load(filePath, res_type='kaiser_fast') 
        mfcc=mfcc_feature_extractor(audio,sampleRate)
        # contrast = contrast_feature_extractor(audio,sampleRate)
        # tonnetz = tonnetz_feature_extractor(audio,sampleRate)
        # centroid = centroid_feature_extractor(audio,sampleRate)
        extractedFeatures.append([mfcc,finalClassLabels])

### creating dictionary with dirPath and label

In [102]:
dict = {"anwar":"C:/Users/EXTRA/DSP_Task3/backend/files/Anwar","aya":"C:/Users/EXTRA/DSP_Task3/backend/files/Aya","ehab":"C:/Users/EXTRA/DSP_Task3/backend/files/Ehab","zeyad":"C:/Users/EXTRA/DSP_Task3/backend/files/Zeyad"}

### creating data frame from our data

In [103]:
extractedFeatures = []
for i in dict.keys():
    add_features(extractedFeatures,dict[i],i)

In [118]:
data=pd.DataFrame(extractedFeatures,columns=['mfcc','class'])
print(data.shape)
data.head()

(400, 2)


,mfcc,class
0,"[-506.52463, 61.64891, 8.294457, 6.1529727, 16...",anwar
1,"[-484.83054, 57.63201, 19.381887, 17.619091, 1...",anwar
2,"[-375.71805, 95.867195, -0.665796, 31.796244, ...",anwar
3,"[-351.94073, 53.247982, 12.9672785, 10.35929, ...",anwar
4,"[-442.27194, 81.52423, 17.646692, 13.475121, 1...",anwar


### splitting the data into features and target

In [105]:
features=data.iloc[:,0:-1]
target=data['class']
features.head()

,mfcc
0,"[-506.52463, 61.64891, 8.294457, 6.1529727, 16..."
1,"[-484.83054, 57.63201, 19.381887, 17.619091, 1..."
2,"[-375.71805, 95.867195, -0.665796, 31.796244, ..."
3,"[-351.94073, 53.247982, 12.9672785, 10.35929, ..."
4,"[-442.27194, 81.52423, 17.646692, 13.475121, 1..."


In [106]:
mfcc=np.array(features['mfcc'].tolist())
# contrast = np.array(features['contrast'].tolist())
# tonnetz = np.array(features['tonnetz'].tolist())
# centroid = np.array(features['centroid'].tolist())

In [107]:
features=[]
for i in range(len(mfcc)):
    features.append(mfcc[i])
data=pd.DataFrame(features)
features = data.values.tolist()
# for i in range(len(features)):
#     features[i] = np.concatenate((features[i][0],features[i][1],features[i][2],features[i][3]))

### encoding the target

In [108]:
encoder = LabelEncoder()
target = encoder.fit_transform(target)

### splitting the data into train and test

In [109]:
xTrain, xTest, yTrain, yTest = train_test_split(features, target, test_size=0.25, random_state=0)

### creating SVM model

In [110]:
classifier= SVC(kernel='linear',decision_function_shape="ovo") 
classifier.fit(xTrain,yTrain)

SVC(decision_function_shape='ovo', kernel='linear')

### predicting xTest

In [111]:
preds = classifier.predict(xTest)

In [112]:
def prediction(pred):
    if pred==0:
        print("anwar")
    elif pred==1:
        print("aya")
    elif pred==2:
        print("ehab")
    elif pred==3:
        print("zeyad")

### making classification report to see the result of the predicton 

In [113]:
print(classification_report(yTest,preds))

              precision    recall  f1-score   support

           0       1.00      0.93      0.96        27
           1       0.97      1.00      0.98        28
           2       1.00      1.00      1.00        19
           3       0.96      1.00      0.98        26

    accuracy                           0.98       100
   macro avg       0.98      0.98      0.98       100
weighted avg       0.98      0.98      0.98       100



In [114]:
testFeatures = features_extractor("C:/Users/EXTRA/DSP_Task3/backend/files/Aya/voice11110622.wav")
pred = classifier.predict(testFeatures)
prediction(pred)


aya


### creating pickle

In [115]:
pickle_out = open("model.pkl", "wb")
pickle.dump(classifier, pickle_out)
pickle_out.close()